## 1. Carga y Procesamiento de Documentos

In [1]:
from unstructured.partition.auto import partition 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from collections import defaultdict
import torch

In [2]:
def unstructured_extraction(pdf_path:str):
    elements = partition(filename=pdf_path)
    return elements

pdf_path = r"C:/Users/danie/Downloads/PaperMind/backend/data/raw/n28a12.pdf"
elements = unstructured_extraction(pdf_path)

## 2. Chunking con Metadatos

In [3]:
pages = defaultdict(list)
for el in elements:
    if el.text.strip():
        metadata = el.metadata.to_dict()
        page = metadata.get('page_number')
        pages[page].append({
            'Texto': el.text.strip(),
            'Nombre del documento': metadata.get('filename'),
            'Idioma': metadata.get('languages', ['unknown'])[0],
        })
all_chunks = []
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
for page_number, elementos in pages.items():
    full_text = " " .join(el['Texto'] for el in elementos)
    base_metadata = {
        'Número de pagina': page_number,
        'Nombre del documento': elementos[0]['Nombre del documento'],
        'Idioma': elementos[0]['Idioma'],
    }
    doc = Document(page_content=full_text, metadata=base_metadata)
    chunks = splitter.split_documents([doc])
    all_chunks.extend(chunks)

## 3. Vector Store y Retriever

In [4]:
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
embeddings_model = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-mpnet-base-v2',
    model_kwargs={'device': device}
)

vector_store = Chroma.from_documents(all_chunks, embeddings_model)

In [5]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

bm25_retriever = BM25Retriever.from_documents(all_chunks, k=5)
chroma_retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

retriever = EnsembleRetriever(
    retrievers=[chroma_retriever, bm25_retriever],
    weights=[0.5, 0.5]
)

## 4. Inicialización de Clientes y Modelos

In [18]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
from groq import Groq
from openai import OpenAI

load_dotenv('.env')

groq_client = Groq(api_key=os.getenv("GROQ_API_KEY"))
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
open_router_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENAI_KEY")
)

gemini_model = genai.GenerativeModel("gemini-2.0-flash")
llama_model_name = "llama-3.3-70b-versatile"
qwen_model_name = "qwen/qwen3-32b"

## 5. Generación de Respuestas

In [7]:
def generate_response(client, model_name, user_query, context_str, client_type):
    system_prompt = "Eres un asistente que responde preguntas basándote únicamente en el contexto proporcionado. Si la respuesta no está en el contexto, di que no tienes suficiente información."
    full_prompt = f"Contexto:\n{context_str}\n\nPregunta: {user_query}"

    try:
        if client_type == 'gemini':
            response = client.generate_content(full_prompt)
            return response.text
        else:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": full_prompt}
            ]
            if client_type == 'groq':
                chat_completion = client.chat.completions.create(messages=messages, model=model_name)
            elif client_type == 'openrouter':
                chat_completion = client.chat.completions.create(messages=messages, model=model_name)
            return chat_completion.choices[0].message.content
    except Exception as e:
        return f"Error al generar respuesta con {model_name}: {e}"

## 6. Ejecución y Comparación

In [19]:
from IPython.display import display, Markdown

question = "Realizame un resumen del documento"

retrieved_docs = retriever.invoke(question)
context_str = "\n\n".join([doc.page_content for doc in retrieved_docs])

response_gemini = generate_response(gemini_model, 'gemini-2.0-flash', question, context_str, 'gemini')
display(Markdown(f"**Respuesta de Gemini:**\n{response_gemini}"))

response_llama = generate_response(groq_client, llama_model_name, question, context_str, 'groq')
display(Markdown(f"**Respuesta de Llama (Groq):**\n{response_llama}"))

response_qwen = generate_response(open_router_client, qwen_model_name, question, context_str, 'openrouter')
display(Markdown(f"**Respuesta de Qwen (OpenRouter):**\n{response_qwen}"))

**Respuesta de Gemini:**
Este documento trata sobre el desarrollo de un corpus (un conjunto de textos) de ensayos académicos revisados.

**Puntos clave:**

*   Los participantes escribieron ensayos que fueron revisados en múltiples borradores (Draft 1, Draft 2 y Draft 3).
*   Se utilizó una interfaz de computadora para resaltar las diferencias entre los borradores iniciales y las revisiones.
*   El corpus contiene 180 ensayos en total.
*   Se alinearon las oraciones entre los borradores y se etiquetaron los propósitos de las revisiones.

Además, el documento hace referencia a otros estudios y datos relacionados con temas como la inversión extranjera directa (FDI), el desarrollo económico y las emisiones contaminantes, mencionando a autores como Chase-Dunn, Tanzi, Ohlin, Bekun, Agosin & Machado entre otros y entidades como las Naciones Unidas y la FAO.


**Respuesta de Llama (Groq):**
El documento parece ser una recopilación de diferentes secciones y referencias de artículos y publicaciones académicas. A continuación, te presento un resumen de los temas y puntos clave que se mencionan:

1. **Desarrollo de un corpus**: Se describe un procedimiento para mejorar la calidad de ensayos a través de revisiones y creación de borradores. Se menciona que se alignmentaron oraciones a lo largo de los borradores y se etiquetaron los propósitos de revisión.
2. **Estadísticas descriptivas**: Se proporcionan estadísticas sobre un conjunto de datos que incluye 180 ensayos, con un promedio de 400 palabras en los dos primeros borradores y 500 palabras en el tercer borrador.
3. **Bibliografía y referencias**: Se incluyen referencias a varios artículos y publicaciones académicas sobre temas como la economía, el desarrollo sostenible, la globalización y la política económica.
4. **Modelos económicos**: Se menciona el modelo de Agosin y Machado (2005) para el estudio de la relación entre la inversión extranjera directa (IED) y la inversión doméstica.
5. **Política y recomendaciones**: Se incluye una sección sobre recomendaciones de política y se menciona que los datos subyacentes están disponibles para garantizar la reproducibilidad de los resultados.
6. **Desarrollo y energía**: Se abordan temas relacionados con el desarrollo sostenible, la energía renovable y la mitigación de emisiones en países como la India.

En general, el documento parece ser una recopilación de diferentes temas y referencias relacionadas con la economía, el desarrollo sostenible y la política económica, con un enfoque en la inversión extranjera directa y el desarrollo de modelos económicos.

**Respuesta de Qwen (OpenRouter):**
El documento presenta información sobre **dos líneas temáticas principales**:

1. **Desarrollo de un corpus de ensayos académicos**:  
   - Se describe un proceso de revisión iterativo en el que participantes mejoran sus ensayos a través de tres borradores (Draft1, Draft2, Draft3).  
   - Se emplea una interfaz computacional para resaltar diferencias entre borradores y alinear frases revisadas, etiquetando los propósitos de las revisiones.  
   - Estadísticas descriptivas: 180 ensayos en total (120 con 400 palabras en promedio para los primeros dos borradores y 60 con 500 palabras para el tercero).

2. **Análisis económico y sostenibilidad**:  
   - Se citan estudios sobre temas como inversión extranjera directa (FDI), desarrollo económico, cambio climático y reforma fiscal, con enfoque en países en desarrollo y África subsahariana.  
   - Se mencionan modelos teóricos (ej.: Agosin & Machado, 2005) para analizar relación entre FDI y estimación de inversión, usando datos de la FAOSTAT y clasificaciones de las Naciones Unidas.  
   - Regiones y autores relevantes: India, África subsahariana, y referencias a trabajos de C. K. Chase-Dunn, V. Tanzi, y F. Bekun.  

**Fuente de datos**: Se utilizan datos de la FAO y bases de la ONU (2020) para apoyar análisis y conclusiones.  
**Aspectos destacados**: Uso de subtítulos como "Conclusiones" en secciones de políticas, y verificación de la reproducibilidad de resultados mediante evidencia respaldada por datos.  

*No se puede identificar la respuesta directa a la solicitud de resumen en la pontencia debido a que la consulta no se dirige específicamente a uno de los contextos proporcionados. El resumen sintetiza los temas principales del documento fragmentado.*

In [9]:
from datasets import load_dataset

peer_qa = load_dataset(
    "UKPLab/PeerQA",
    "qa",
)

peer_papers = load_dataset(
    "UKPLab/PeerQA",
    "papers",
    split='test'
)

Using the latest cached version of the dataset since UKPLab/PeerQA couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'qa' at C:\Users\danie\.cache\huggingface\datasets\UKPLab___peer_qa\qa\1.0.0\2b4c608f2e508bafc5d874167212597ed9d3351a9f107dfa83f628a4bdfbc337 (last modified on Sun Jul 20 11:03:10 2025).
Using the latest cached version of the dataset since UKPLab/PeerQA couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'papers' at C:\Users\danie\.cache\huggingface\datasets\UKPLab___peer_qa\papers\1.0.0\2b4c608f2e508bafc5d874167212597ed9d3351a9f107dfa83f628a4bdfbc337 (last modified on Tue Jul 15 20:49:57 2025).


In [32]:
from datasets import load_dataset

# Dataset de preguntas + respuestas
peerqa_ds = load_dataset("UKPLab/PeerQA", "qa", split="test")

# Dataset de papers (documentos fuente)
papers_ds = load_dataset("UKPLab/PeerQA", "papers", split="test")


Using the latest cached version of the dataset since UKPLab/PeerQA couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'qa' at C:\Users\danie\.cache\huggingface\datasets\UKPLab___peer_qa\qa\1.0.0\2b4c608f2e508bafc5d874167212597ed9d3351a9f107dfa83f628a4bdfbc337 (last modified on Sun Jul 20 11:03:10 2025).
Using the latest cached version of the dataset since UKPLab/PeerQA couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'papers' at C:\Users\danie\.cache\huggingface\datasets\UKPLab___peer_qa\papers\1.0.0\2b4c608f2e508bafc5d874167212597ed9d3351a9f107dfa83f628a4bdfbc337 (last modified on Tue Jul 15 20:49:57 2025).


## 6. Preparación de los datos para la evaluación

In [34]:
from datasets import load_dataset, Dataset
import math
import pandas as pd

# Cargar dataset
qa = load_dataset("UKPLab/PeerQA", "qa")["test"]

# Función mejorada para validar el ground truth
def is_valid_text(val):
    # Verificar si es None
    if val is None:
        return False
    
    # Verificar si es NaN float
    if isinstance(val, float) and math.isnan(val):
        return False
    
    # Verificar si es pandas NaN
    if pd.isna(val):
        return False
    
    # Convertir a string y verificar
    val_str = str(val).strip().lower()
    
    # Verificar si es string vacío o variaciones de "nan"
    if val_str == "" or val_str in ["nan", "none", "null", "<na>", "n/a"]:
        return False
    
    return True

# Extraer ground truth con prioridad y validar
def get_ground_truth(example):
    for key in ["answer_free_form_augmented", "answer_free_form"]:
        val = example.get(key)
        if is_valid_text(val):
            return val
    return None

# Función para validar contextos
def is_valid_context(context):
    if not context:
        return False
    if isinstance(context, list):
        # Verificar que la lista no esté vacía y tenga elementos válidos
        return len(context) > 0 and all(is_valid_text(item) for item in context)
    return is_valid_text(context)

# Construir dataset filtrado con más debugging
examples = []
total_examples = len(qa)
filtered_stats = {
    "not_answerable": 0,
    "invalid_paper_id": 0,
    "no_ground_truth": 0,
    "invalid_context": 0,
    "valid": 0
}

print(f"Procesando {total_examples} ejemplos...")

for i, ex in enumerate(qa):
    # Debug cada 1000 ejemplos
    if i % 1000 == 0:
        print(f"Procesando ejemplo {i}/{total_examples}")
    
    # Verificar si es answerable
    if not ex["answerable"]:
        filtered_stats["not_answerable"] += 1
        continue

    # Verificar paper_id
    if not is_valid_text(ex["paper_id"]):
        filtered_stats["invalid_paper_id"] += 1
        continue
    
    # Obtener ground truth
    ground_truth = get_ground_truth(ex)
    if not ground_truth:
        filtered_stats["no_ground_truth"] += 1
        # Debug: imprimir algunos casos problemáticos
        if filtered_stats["no_ground_truth"] <= 5:
            print(f"Ejemplo sin ground_truth válido:")
            print(f"  answer_free_form_augmented: {repr(ex.get('answer_free_form_augmented'))}")
            print(f"  answer_free_form: {repr(ex.get('answer_free_form'))}")
        continue
    
    # Verificar contexto
    context = ex["answer_evidence_sent"] or ex["raw_answer_evidence"]
    if not is_valid_context(context):
        filtered_stats["invalid_context"] += 1
        continue
    
    # Si llegamos aquí, el ejemplo es válido
    filtered_stats["valid"] += 1
    examples.append({
        "question": ex["question"],
        "ground_truth": ground_truth,
        "contexts": context,
        "paper_id": ex["paper_id"]
    })

# Crear dataset limpio
qa_clean_dataset = Dataset.from_list(examples)

print(f"\nNúmero final de ejemplos válidos: {len(qa_clean_dataset)}")

Using the latest cached version of the dataset since UKPLab/PeerQA couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'qa' at C:\Users\danie\.cache\huggingface\datasets\UKPLab___peer_qa\qa\1.0.0\2b4c608f2e508bafc5d874167212597ed9d3351a9f107dfa83f628a4bdfbc337 (last modified on Sun Jul 20 11:03:10 2025).


Procesando 579 ejemplos...
Procesando ejemplo 0/579
Ejemplo sin ground_truth válido:
  answer_free_form_augmented: 'nan'
  answer_free_form: None
Ejemplo sin ground_truth válido:
  answer_free_form_augmented: 'nan'
  answer_free_form: None
Ejemplo sin ground_truth válido:
  answer_free_form_augmented: 'nan'
  answer_free_form: None
Ejemplo sin ground_truth válido:
  answer_free_form_augmented: 'nan'
  answer_free_form: None
Ejemplo sin ground_truth válido:
  answer_free_form_augmented: 'nan'
  answer_free_form: None

Número final de ejemplos válidos: 267


## 7. Clase orquestadora de la evaluación por lotes

In [36]:
import json
import os
from datasets import load_dataset, Dataset
import datetime
import time


class RAGEvaluationGenerator:
    """
    Clase para generar respuestas y dataset RAGAS usando RAGPipeline.
    Maneja indexación de documentos y procesamiento en lotes.
    """
    def __init__(self, rag_pipeline, clean_dataset, batch_size=3, delay=30):
        """
        Args:
            rag_pipeline: Instancia de RAGPipeline.
            clean_dataset: Dataset limpio (qa_clean_dataset).
            batch_size: Tamaño del lote para procesamiento.
            delay: Segundos de espera entre lotes.
        """
        self.rag = rag_pipeline
        self.dataset = clean_dataset
        self.batch_size = batch_size
        self.delay = delay
        self.results_file = "rag_evaluation_results.json"
        self.results = self.load_previous_results()
        print(f"✅ Inicializado RAGEvaluationGenerator con {len(clean_dataset)} ejemplos")

    def load_previous_results(self):
        """Cargar resultados previos para reanudar."""
        if os.path.exists(self.results_file):
            with open(self.results_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        return []

    def save_results(self):
        """Guardar resultados incrementalmente."""
        with open(self.results_file, 'w', encoding='utf-8') as f:
            json.dump(self.results, f, ensure_ascii=False, indent=2)

    def setup_retriever(self):
        """Configurar el retriever PeerQA (equivalente a index_documents)."""
        print("Configurando retriever PeerQA...")
        try:
            # Tu método ya no retorna nada, solo configura self.rag.retriever
            self.rag.setup_peerqa_retriever()
            print("✅ Retriever configurado exitosamente")
            return True
        except Exception as e:
            print(f"❌ Error configurando retriever: {str(e)}")
            return False

    def generate_responses_batch(self, model_type='groq', start_idx=None, max_samples=30):
        """
        Generar respuestas en lotes.
        
        Args:
            model_type: 'groq', 'gemini' o 'openrouter'.
            start_idx: Índice inicial (None para continuar desde resultados previos).
            max_samples: Máximo número de muestras a procesar.
        """
        # Verificar que el retriever esté configurado
        if not self.rag.retriever:
            print("⚠️ Retriever no configurado. Configurando automáticamente...")
            if not self.setup_retriever():
                raise ValueError("No se pudo configurar el retriever")

        total_samples = len(self.dataset)
        if start_idx is None:
            start_idx = len(self.results)

        end_idx = min(start_idx + max_samples, total_samples) if max_samples else total_samples

        print(f"\n🚀 Generando respuestas con modelo: {model_type}")
        print(f"📋 Procesando desde índice {start_idx} hasta {end_idx}")
        print(f"📦 Lotes de {self.batch_size} con {self.delay}s de espera")
        print(f"📊 Ya procesados: {len(self.results)} ejemplos")

        for i in range(start_idx, end_idx, self.batch_size):
            batch_end = min(i + self.batch_size, end_idx)
            current_batch = range(i, batch_end)

            print(f"\n🔄 Procesando lote {i//self.batch_size + 1}: índices {i}-{batch_end-1}")

            for idx in current_batch:
                try:
                    example = self.dataset[idx]
                    question = example['question']
                    print(f"[{idx+1}/{end_idx}] Procesando: {question[:50]}...")

                    # Generar respuesta con RAGPipeline
                    rag_result = self.rag.query_rag(
                        question=question, 
                        model_type=model_type,
                        max_context_length=3000
                    )

                    # Preparar resultado para RAGAS
                    result = {
                        'question': question,
                        'answer': rag_result['answer'],
                        'contexts': rag_result['contexts'],  # Ya viene limitado a top 5
                        'ground_truth': example['ground_truth'],
                        'paper_id': example.get('paper_id', 'unknown'),
                        'model_used': model_type,
                        'timestamp': datetime.datetime.now().isoformat(),
                        'processed_idx': idx,
                        'context_used_length': len(rag_result['context_used']),
                        'retrieved_docs_count': len(rag_result['retrieved_docs'])
                    }
                    self.results.append(result)
                    print(f"  ✅ Completado! (Contexto: {result['context_used_length']} chars)")

                except Exception as e:
                    print(f"❌ Error en índice {idx}: {str(e)}")
                    error_result = {
                        'question': self.dataset[idx]['question'] if idx < len(self.dataset) else "Error loading",
                        'answer': f"ERROR: {str(e)}",
                        'contexts': [],
                        'ground_truth': self.dataset[idx].get('ground_truth', 'N/A') if idx < len(self.dataset) else "N/A",
                        'paper_id': self.dataset[idx].get('paper_id', 'N/A') if idx < len(self.dataset) else "N/A",
                        'model_used': model_type,
                        'timestamp': datetime.datetime.now().isoformat(),
                        'processed_idx': idx,
                        'error': True
                    }
                    self.results.append(error_result)

            # Guardar progreso después de cada lote
            self.save_results()
            print(f"💾 Progreso guardado: {len(self.results)} ejemplos completados")

            # Esperar entre lotes (excepto en el último)
            if batch_end < end_idx:
                print(f"⏳ Esperando {self.delay} segundos antes del siguiente lote...")
                time.sleep(self.delay)

        print(f"\n🎉 ¡Proceso completado! Total: {len(self.results)} ejemplos procesados")
        return self.results

    def get_ragas_dataset(self, filter_errors=True):
        """
        Crear dataset para RAGAS.
        
        Args:
            filter_errors: Excluir ejemplos con errores.
        """
        clean_results = [r for r in self.results if not r.get('error', False)] if filter_errors else self.results

        if not clean_results:
            print("❌ No hay resultados válidos para crear dataset")
            return None

        ragas_data = {
            'question': [r['question'] for r in clean_results],
            'answer': [r['answer'] for r in clean_results],
            'contexts': [r['contexts'] for r in clean_results],
            'ground_truth': [r['ground_truth'] for r in clean_results]
        }

        dataset = Dataset.from_dict(ragas_data)
        print(f"📊 Dataset RAGAS creado exitosamente: {len(dataset)} ejemplos")
        return dataset

    def get_summary(self):
        """Resumen del progreso actual."""
        if not self.results:
            return "📭 No hay resultados aún"

        total = len(self.results)
        errors = sum(1 for r in self.results if r.get('error', False))
        successful = total - errors
        
        # Contar modelos usados
        models_used = {}
        for r in self.results:
            model = r.get('model_used', 'unknown')
            models_used[model] = models_used.get(model, 0) + 1

        # Estadísticas adicionales
        avg_context_length = 0
        if successful > 0:
            context_lengths = [r.get('context_used_length', 0) for r in self.results if not r.get('error', False)]
            avg_context_length = sum(context_lengths) / len(context_lengths) if context_lengths else 0

        return f"""
📊 RESUMEN DEL PROGRESO:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📈 Total procesado: {total}
✅ Exitosos: {successful}
❌ Con errores: {errors}
🤖 Modelos usados: {dict(models_used)}
📚 Dataset original: {len(self.dataset)} ejemplos
📊 Progreso: {(total/len(self.dataset)*100):.1f}%
🎯 Contexto promedio: {avg_context_length:.0f} caracteres
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        """

    def get_failed_examples(self):
        """Obtener ejemplos que fallaron para debugging."""
        failed = [r for r in self.results if r.get('error', False)]
        if not failed:
            return "✅ No hay ejemplos fallidos"
        
        return f"❌ {len(failed)} ejemplos fallidos:\n" + "\n".join([
            f"  - Índice {r['processed_idx']}: {r['answer'][:100]}..."
            for r in failed[:5]  # Mostrar solo los primeros 5
        ])

    def export_results(self, filename=None):
        """Exportar resultados a un archivo específico."""
        if filename is None:
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"rag_evaluation_export_{timestamp}.json"
        
        export_data = {
            'metadata': {
                'total_examples': len(self.results),
                'successful': len([r for r in self.results if not r.get('error', False)]),
                'failed': len([r for r in self.results if r.get('error', False)]),
                'export_timestamp': datetime.datetime.now().isoformat(),
                'dataset_size': len(self.dataset)
            },
            'results': self.results
        }
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(export_data, f, ensure_ascii=False, indent=2)
        
        print(f"📤 Resultados exportados a: {filename}")
        return filename

## 8. Clase de la RAG intermedia preparada y adpatada para la evaluación

In [35]:
import os
from dotenv import load_dotenv
from groq import Groq
import google.generativeai as genai
from openai import OpenAI
from collections import defaultdict
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from unstructured.partition.auto import partition
from datasets import load_dataset
import pickle
import chromadb
from chromadb.config import Settings

class RAGPipeline:
    def __init__(self, chroma_host="localhost", chroma_port=8000):
        load_dotenv('../.env')
        self.groq_client = Groq(api_key=os.getenv("GROQ_API_KEY"))
        genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
        self.gemini_model = genai.GenerativeModel("gemini-2.0-flash")
        self.open_router_client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=os.getenv("OPENROUTE_API_KEY")
        )
        self.llama_model_name = "llama-3.3-70b-versatile"
        self.qwen_model_name = "qwen/qwen2-72b-instruct"
        self.retriever = None
        
        # Configuración de Chroma Client para Docker
        try:
            self.chroma_client = chromadb.HttpClient(
                host=chroma_host,
                port=chroma_port,
                settings=Settings(allow_reset=True)
            )
            # Test de conectividad
            self.chroma_client.heartbeat()
            print(f"✅ Conectado a Chroma en {chroma_host}:{chroma_port}")
        except Exception as e:
            print(f"❌ Error conectando a Chroma: {e}")
            print(f"💡 Asegúrate de que Chroma esté corriendo en {chroma_host}:{chroma_port}")
            print("   Comando: docker run -d --name chroma-rag -p 8000:8000 -v chroma_rag_data:/chroma/chroma chromadb/chroma:latest")
            raise
        
        self.collection_name = "peerqa_papers"

    def setup_peerqa_retriever(self):
        """Configura el retriever específicamente para PeerQA usando Chroma Docker"""
        chunks_path = "../data/peerqa_chunks.pkl"
        
        # Verificar si ya existe la colección en Chroma
        try:
            collection = self.chroma_client.get_collection(name=self.collection_name)
            collection_count = collection.count()
            print(f"Colección '{self.collection_name}' ya existe con {collection_count} documentos")
            
            # Si existe la colección y tenemos chunks guardados, cargar directamente
            if collection_count > 0 and os.path.exists(chunks_path):
                print("Cargando chunks desde archivo...")
                with open(chunks_path, "rb") as f:
                    chunks = pickle.load(f)
                
                self.retriever = self.crear_retriever_desde_chunks(chunks)
                print("Retriever cargado correctamente desde datos existentes!")
                return self.retriever
                
        except Exception as e:
            print(f"Colección no existe o error: {e}")
            print("Creando nueva colección...")
        
        # Procesar dataset si no existe
        print("Procesando dataset PeerQA...")
        dataset = load_dataset("UKPLab/PeerQA", "papers")
        papers = dataset['test']
        print("Procesando papers a chunks...")
        chunks = self.procesar_dataset_peerqa_a_chunks_optimizado(papers)

        # Guardar chunks en disco
        print(f"Guardando {len(chunks)} chunks en disco...")
        os.makedirs(os.path.dirname(chunks_path), exist_ok=True)
        with open(chunks_path, "wb") as f:
            pickle.dump(chunks, f)

        print(f"Total chunks: {len(chunks)}")
        print("Creando retriever...")
        self.retriever = self.crear_retriever_desde_chunks(chunks)
        print("Retriever configurado correctamente!")
        return self.retriever

    def crear_retriever_desde_chunks(self, chunks):
        """Crear retriever usando Chroma Docker"""
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"Usando device: {device}")

        embeddings_model = HuggingFaceEmbeddings(
            model_name='sentence-transformers/all-mpnet-base-v2',
            model_kwargs={'device': device},
            encode_kwargs={'normalize_embeddings': True}
        )

        # Crear vector store con Chroma Docker
        print("Configurando vector store con Chroma Docker...")
        vector_store = Chroma(
            client=self.chroma_client,
            collection_name=self.collection_name,
            embedding_function=embeddings_model
        )
        
        # Verificar si la colección ya tiene documentos
        try:
            collection = self.chroma_client.get_collection(name=self.collection_name)
            if collection.count() == 0:
                print("Agregando documentos a la colección...")
                vector_store.add_documents(chunks)
                print(f"Agregados {len(chunks)} documentos a Chroma")
            else:
                print(f"La colección ya tiene {collection.count()} documentos")
        except:
            print("Creando colección y agregando documentos...")
            vector_store.add_documents(chunks)
            print(f"Agregados {len(chunks)} documentos a Chroma")

        # Crear BM25 retriever
        print("Creando BM25 retriever...")
        bm25_retriever = BM25Retriever.from_documents(chunks, k=7)
        
        # Crear vector retriever
        print("Creando vector retriever...")
        chroma_retriever = vector_store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 7}
        )

        # Crear ensemble retriever
        print("Creando ensemble retriever...")
        retriever = EnsembleRetriever(
            retrievers=[chroma_retriever, bm25_retriever],
            weights=[0.7, 0.3]
        )

        return retriever

    def procesar_dataset_peerqa_a_chunks_optimizado(self, papers_dataset):
        """Versión optimizada para PeerQA con chunks más grandes y mejor metadata"""
        
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=512,  
            chunk_overlap=150,  
            separators=["\n\n", "\n", ". ", " ", ""]  
        )
        
        all_chunks = []
        papers_by_id = {}
        
        # Agrupar por paper_id
        for row in papers_dataset:
            paper_id = row["paper_id"]
            if paper_id not in papers_by_id:
                papers_by_id[paper_id] = []
            papers_by_id[paper_id].append(row)
        
        for paper_id, sections in papers_by_id.items():
            # Construir texto completo del paper
            full_paper_text = ""
            section_info = []
            
            for section in sections:
                content = section["content"].strip()
                if content:
                    section_type = section.get("type", "text")
                    last_heading = section.get("last_heading", "")
                    
                    if section_type in ["title", "heading"] or last_heading:
                        content = f"[{section_type.upper()}] {content}"
                    
                    full_paper_text += f"{content}\n\n"
                    section_info.append({
                        "type": section_type,
                        "heading": last_heading,
                        "idx": section.get("idx", 0)
                    })
            
            # Crear chunks para este paper
            if full_paper_text.strip():
                doc = Document(
                    page_content=full_paper_text.strip(),
                    metadata={
                        "paper_id": paper_id,
                        "source": "peerqa",
                        "sections_count": len(sections)
                    }
                )
                
                chunks = splitter.split_documents([doc])
                
                # Agregar metadata específico a cada chunk
                for i, chunk in enumerate(chunks):
                    chunk.metadata.update({
                        "paper_id": paper_id,
                        "chunk_id": f"{paper_id}_{i}",
                        "source": "peerqa",
                        "chunk_index": i,
                        "total_chunks": len(chunks)
                    })
                
                all_chunks.extend(chunks)
        
        print(f"Procesados {len(papers_by_id)} papers únicos en {len(all_chunks)} chunks")
        return all_chunks

    def query_rag(self, question, model_type='groq', max_context_length=3000):
        """Función principal para queries con límite de contexto"""
        if not self.retriever:
            raise ValueError("Retriever no configurado. Ejecuta setup_peerqa_retriever() primero.")
        
        # Recuperar documentos relevantes
        retrieved_docs = self.retriever.invoke(question)
        
        # Construir contexto respetando límite
        context_parts = []
        total_length = 0
        
        for doc in retrieved_docs:
            doc_text = doc.page_content
            if total_length + len(doc_text) <= max_context_length:
                context_parts.append(f"Paper: {doc.metadata.get('paper_id', 'unknown')}\n{doc_text}")
                total_length += len(doc_text)
            else:
                # Truncar si es necesario
                remaining = max_context_length - total_length
                if remaining > 100:  
                    truncated = doc_text[:remaining] + "..."
                    context_parts.append(f"Paper: {doc.metadata.get('paper_id', 'unknown')}\n{truncated}")
                break
        
        context_str = "\n\n---\n\n".join(context_parts)
        
        if model_type == 'groq':
            response = self.generate_response(
                self.groq_client, self.llama_model_name, question, context_str, 'groq'
            )
        elif model_type == 'gemini':
            response = self.generate_response(
                self.gemini_model, None, question, context_str, 'gemini'
            )
        elif model_type == 'openrouter':
            response = self.generate_response(
                self.open_router_client, self.qwen_model_name, question, context_str, 'openrouter'
            )
        else:
            raise ValueError("model_type debe ser 'groq', 'gemini' o 'openrouter'")
        
        return {
            'question': question,
            'answer': response,
            'contexts': [doc.page_content for doc in retrieved_docs[:5]],  # Solo top 5 para RAGAS
            'retrieved_docs': retrieved_docs,
            'context_used': context_str
        }

    def generate_response(self, client, model_name, user_query, context_str, client_type):
        """Prompt mejorado para evaluación académica"""
        system_prompt = """Eres un asistente de investigación experto en machine learning y procesamiento de lenguaje natural. 

INSTRUCCIONES:
- Responde basándote únicamente en el contexto proporcionado de los papers académicos
- Si tienes información suficiente, proporciona una respuesta completa y técnicamente precisa
- Cita conceptos, métodos o resultados específicos cuando sea relevante
- Si la información es parcial, responde con lo disponible y especifica qué no puedes determinar
- Solo di "La información proporcionada no es suficiente" si el contexto es completamente irrelevante
- Usa terminología técnica apropiada
- Sé conciso pero completo"""

        full_prompt = f"Contexto de papers académicos:\n{context_str}\n\nPregunta: {user_query}\n\nRespuesta basada en el contexto:"
        
        try:
            if client_type == 'gemini':
                response = client.generate_content(full_prompt)
                return response.text
            else:
                messages = [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": full_prompt}
                ]
                if client_type == 'groq':
                    chat_completion = client.chat.completions.create(
                        messages=messages, 
                        model=model_name,
                        temperature=0.1,  
                        max_tokens=1000
                    )
                elif client_type == 'openrouter':
                    chat_completion = client.chat.completions.create(
                        messages=messages, 
                        model=model_name,
                        temperature=0.1,
                        max_tokens=1000
                    )
                return chat_completion.choices[0].message.content
        except Exception as e:
            return f"Error al generar respuesta: {str(e)}"

    def reset_collection(self):
        """Método para resetear la colección si es necesario"""
        try:
            self.chroma_client.delete_collection(name=self.collection_name)
            print(f"Colección '{self.collection_name}' eliminada")
        except Exception as e:
            print(f"Error al eliminar colección: {e}")

    def get_collection_info(self):
        """Método para obtener información sobre la colección"""
        try:
            collection = self.chroma_client.get_collection(name=self.collection_name)
            count = collection.count()
            print(f"Colección '{self.collection_name}' tiene {count} documentos")
            return count
        except Exception as e:
            print(f"Error al obtener información de la colección: {e}")
            return 0

    def procesar_pdf_a_chunks(self, pdf_path):
        """Método original para PDFs (mantenido para compatibilidad)"""
        elements = partition(filename=pdf_path)
        pages = defaultdict(list)
        for el in elements:
            if el.text.strip():
                metadata = el.metadata.to_dict()
                page = metadata.get('page_number')
                pages[page].append({
                    'Texto': el.text.strip(),
                    'Nombre del documento': metadata.get('filename'),
                    'Idioma': metadata.get('languages', ['unknown'])[0],
                })
        all_chunks = []
        splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        for page_number, elementos in pages.items():
            full_text = " ".join(el['Texto'] for el in elementos)
            base_metadata = {
                'Número de pagina': page_number,
                'Nombre del documento': elementos[0]['Nombre del documento'],
                'Idioma': elementos[0]['Idioma'],
            }
            doc = Document(page_content=full_text, metadata=base_metadata)
            chunks = splitter.split_documents([doc])
            all_chunks.extend(chunks)
        return all_chunks

In [37]:
# 1. Inicializar tu RAGPipeline
rag = RAGPipeline()  # Sin parámetros Chroma ya que usa archivos locales

# Setup (solo la primera vez, después ya está persistido)
rag.setup_peerqa_retriever()

# 2. Crear el generador (mismo código que tenías)
generator = RAGEvaluationGenerator(
    rag_pipeline=rag,
    clean_dataset=qa_clean_dataset,  # Tu dataset filtrado
    batch_size=3,    # Solo 3 por lote (más conservador)
    delay=30         # 30 segundos entre lotes (no 90)
)

# 3. Procesar de a poquito (empezar con 4 ejemplos de prueba)
generator.generate_responses_batch(
    model_type='gemini',
    max_samples=4  # Solo 4 para probar
)

# 4. Ver progreso
print(generator.get_summary())

# 5. Cuando esté listo, crear dataset para RAGAS
ragas_dataset = generator.get_ragas_dataset()

Using the latest cached version of the dataset since UKPLab/PeerQA couldn't be found on the Hugging Face Hub


✅ Conectado a Chroma en localhost:8000
Colección 'peerqa_papers' ya existe con 12844 documentos
Procesando dataset PeerQA...


Found the latest cached dataset configuration 'papers' at C:\Users\danie\.cache\huggingface\datasets\UKPLab___peer_qa\papers\1.0.0\2b4c608f2e508bafc5d874167212597ed9d3351a9f107dfa83f628a4bdfbc337 (last modified on Tue Jul 15 20:49:57 2025).


Procesando papers a chunks...
Procesados 90 papers únicos en 9732 chunks
Guardando 9732 chunks en disco...
Total chunks: 9732
Creando retriever...
Usando device: cuda
Configurando vector store con Chroma Docker...
La colección ya tiene 12844 documentos
Creando BM25 retriever...
Creando vector retriever...
Creando ensemble retriever...
Retriever configurado correctamente!
✅ Inicializado RAGEvaluationGenerator con 267 ejemplos

🚀 Generando respuestas con modelo: gemini
📋 Procesando desde índice 8 hasta 12
📦 Lotes de 3 con 30s de espera
📊 Ya procesados: 8 ejemplos

🔄 Procesando lote 3: índices 8-10
[9/12] Procesando: Can the proposed algorithm be used to recover real...
  ✅ Completado! (Contexto: 3551 chars)
[10/12] Procesando: Can the pairwise distance of the latent variables ...
  ✅ Completado! (Contexto: 3456 chars)
[11/12] Procesando: Can the parameters of the BLOSUM matrix be estimat...
  ✅ Completado! (Contexto: 3181 chars)
💾 Progreso guardado: 11 ejemplos completados
⏳ Esperando 30